#### Lab assignment - Python notebook for Coursera

In [36]:
# instantiating the libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from pandas import DataFrame
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means for clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

!pip install geocoder
import geocoder


!pip install lxml


print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


Read in Wiki source page for Toronto postal codes

In [2]:
# set variable for URL, using older html
url = "http://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"
t_df = pd.read_html(url, match='Toronto')
    
# create dataframe
df = pd.DataFrame(t_df[0])

Rename columns

In [3]:
# set column names as instructed
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
df.columns=column_names

### Deliverable 1 - Filter Boroughs and Neighborhoods & report out the rows and column using shape function

Here the assumptions are Postals codes may not be assigned to a borough and all neighborhoods must belong to a borough. 

In [4]:
# filter out rows with empty boroughs
df_w_boroughs=df[df['Borough']!='Not assigned']
    
# reset indices
df_w_boroughs.reset_index(drop=True, inplace=True)
df_w_boroughs.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
df_w_boroughs.shape

(103, 3)

Use geocoder to find lat and long on postal codes

In [6]:
pcl_column_names = ['PostalCode', 'Latitude', 'Longitude']
tor_temp1=pd.DataFrame(columns=pcl_column_names)

In [7]:
pc_list=df_w_boroughs['PostalCode']
for data in pc_list:
    lat_lng_coords = None
    #print(data)
    while(lat_lng_coords is None):
    # loop until you get the coordinates
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(data))
        lat_lng_coords = g.json
        lat1 = lat_lng_coords['lat']
        long1 = lat_lng_coords['lng']   

    tor_temp1 = tor_temp1.append({'PostalCode': data, 'Latitude': lat1,'Longitude': long1}, ignore_index=True)

### Deliverable 2 - Merge Postal Code df with Lat and Long variables to existing Postal Code df and print df

In [8]:
toronto_pcl_merged = df_w_boroughs

toronto_pcl_merged = toronto_pcl_merged.join(tor_temp1.set_index('PostalCode'), on='PostalCode')

toronto_pcl_merged.head(12) # checking the columns

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66263,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
7,M3B,North York,Don Mills,43.74923,-79.36186
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


In [9]:
tor_temp2=toronto_pcl_merged[toronto_pcl_merged['Borough'].str.contains("Toronto")]
tor_temp2.reset_index(drop=True, inplace=True)
tor_temp2.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
4,M4E,East Toronto,The Beaches,43.67709,-79.29547
5,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306
6,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493
7,M6G,Downtown Toronto,Christie,43.66869,-79.42071
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.64970,-79.38258
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.66505,-79.43891


Identifying the number of boroughs containing the string "Toronto" in its name

In [10]:
#tor_temp2['Neighborhood'] = tor_temp2.loc[:,'Neighborhood'].str.split (', ')

#tor_temp2.explode('Neighborhood')

tor_temp3=tor_temp2.assign(Neighborhood=tor_temp2['Neighborhood'].str.split(', ')).explode('Neighborhood')

tor_temp3.reset_index(drop=True, inplace=True)

tor_temp3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,43.65512,-79.36264
1,M5A,Downtown Toronto,Harbourfront,43.65512,-79.36264
2,M7A,Downtown Toronto,Queen's Park,43.66253,-79.39188
3,M7A,Downtown Toronto,Ontario Provincial Government,43.66253,-79.39188
4,M5B,Downtown Toronto,Garden District,43.65739,-79.37804
...,...,...,...,...,...
77,M5X,Downtown Toronto,First Canadian Place,43.64828,-79.38146
78,M5X,Downtown Toronto,Underground city,43.64828,-79.38146
79,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
80,M7Y,East Toronto,Business reply mail Processing Centre,43.64869,-79.38544


In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(

        len(tor_temp3['Borough'].unique()),

        len(tor_temp3['Neighborhood'].value_counts())))

The dataframe has 5 boroughs and 79 neighborhoods.


Creating a map of Toronto

In [12]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor_temp3['Latitude'], tor_temp3['Longitude'], tor_temp3['Borough'], tor_temp3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
tor_temp3.loc[0, 'Neighborhood']

'Regent Park'

In [15]:
neighborhood_latitude = tor_temp3.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = tor_temp3.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = tor_temp3.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park are 43.65512000000007, -79.36263999999994.


Using FourSquare API to explore the Toronto boroughs

In [16]:
CLIENT_ID = 'NUVVRJQRBCVLGGQ4POH05J5SPFARUGA30UDP0PN45ABPD13P' # your Foursquare ID
CLIENT_SECRET = '1QODXY1RZY2I1L2BLUNWCPJDEGO5A2GIDXMBX3WHJ5XS0JAU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NUVVRJQRBCVLGGQ4POH05J5SPFARUGA30UDP0PN45ABPD13P
CLIENT_SECRET:1QODXY1RZY2I1L2BLUNWCPJDEGO5A2GIDXMBX3WHJ5XS0JAU


In [17]:
LIMIT = 100 # limit of number of venues
radius = 500 # # radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [18]:
results = requests.get(url).json()
#results

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto


In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [20]:
venues = results['response']['groups'][0]['items']
#venues = results['response']['venues'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,Souvlaki Express,Greek Restaurant,43.655584,-79.364438
4,The Yoga Lounge,Yoga Studio,43.655515,-79.364955


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Execution of the above function on each neighborhood and create a new dataframe called _toronto_venues_. ???? - checkpoint


In [22]:
# type your answer here
toronto_venues=getNearbyVenues(names=tor_temp3['Neighborhood'],
                                latitudes=tor_temp3['Latitude'],
                                longitudes=tor_temp3['Longitude'])

Regent Park
Harbourfront
Queen's Park
Ontario Provincial Government
Garden District
Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond
Adelaide
King
Dufferin
Dovercourt Village
Harbourfront East
Union Station
Toronto Islands
Little Portugal
Trinity
The Danforth West
Riverdale
Toronto Dominion Centre
Design Exchange
Brockton
Parkdale Village
Exhibition Place
India Bazaar
The Beaches West
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede
The Junction
Weston-Pellam Park
Carlton Village
Davisville North
Forest Hill North & West
Forest Hill Road Park
High Park
The Junction South
North Toronto West
Lawrence Park
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
University of Toronto
Harbord
Runnymede
Swansea
Moore Park
Summerhill East
Kensington Market
Chinatown
Grange Park
Summerhill West
Rathnelly
South Hill
Forest Hill SE
Deer Park
CN Tower
King and Spadina
Railway Lands
Harbourfront West
Bathurst Quay
South Ni

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(3673, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,Regent Park,43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
2,Regent Park,43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,Regent Park,43.65512,-79.36264,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
4,Regent Park,43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,77,77,77,77,77,77
Berczy Park,63,63,63,63,63,63
Brockton,84,84,84,84,84,84
Business reply mail Processing Centre,100,100,100,100,100,100
...,...,...,...,...,...,...
Union Station,52,52,52,52,52,52
University of Toronto,52,52,52,52,52,52
Victoria Hotel,100,100,100,100,100,100


In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


Analyze Each Neighborhood

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_onehot.shape

(3673, 231)

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Adelaide,0.000000,0.000000,0.0,0.03,0.000000,0.000000,0.010000,0.0,0.00000,...,0.0,0.0,0.000000,0.010000,0.000000,0.000000,0.0,0.01,0.0,0.000000
1,Bathurst Quay,0.012987,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.012987,0.000000,0.0,0.00,0.0,0.000000
2,Berczy Park,0.015873,0.000000,0.0,0.00,0.015873,0.000000,0.015873,0.0,0.00000,...,0.0,0.0,0.000000,0.015873,0.000000,0.000000,0.0,0.00,0.0,0.000000
3,Brockton,0.011905,0.011905,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.02381,...,0.0,0.0,0.000000,0.011905,0.000000,0.000000,0.0,0.00,0.0,0.000000
4,Business reply mail Processing Centre,0.000000,0.000000,0.0,0.02,0.000000,0.000000,0.010000,0.0,0.01000,...,0.0,0.0,0.000000,0.020000,0.000000,0.000000,0.0,0.01,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Union Station,0.000000,0.000000,0.0,0.00,0.000000,0.038462,0.000000,0.0,0.00000,...,0.0,0.0,0.019231,0.019231,0.000000,0.000000,0.0,0.00,0.0,0.000000
75,University of Toronto,0.019231,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.000000,0.019231,0.000000,0.019231,0.0,0.00,0.0,0.019231
76,Victoria Hotel,0.000000,0.000000,0.0,0.03,0.000000,0.000000,0.010000,0.0,0.00000,...,0.0,0.0,0.000000,0.010000,0.000000,0.000000,0.0,0.01,0.0,0.000000
77,Weston-Pellam Park,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000


In [29]:
toronto_grouped.shape

(79, 231)

Printing each neighborhood along with the top 5 most common venues

In [30]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0                Hotel  0.06
1          Coffee Shop  0.06
2                 Café  0.06
3           Restaurant  0.04
4  Japanese Restaurant  0.04


----Bathurst Quay----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.06
2                Café  0.05
3                 Bar  0.04
4                Park  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.06
1  Seafood Restaurant  0.05
2        Cocktail Bar  0.05
3              Bakery  0.05
4            Beer Bar  0.03


----Brockton----
         venue  freq
0          Bar  0.06
1         Café  0.06
2  Coffee Shop  0.05
3   Restaurant  0.05
4    Gift Shop  0.04


----Business reply mail Processing Centre----
              venue  freq
0       Coffee Shop  0.08
1             Hotel  0.05
2              Café  0.04
3  Asian Restaurant  0.03
4        Restaurant  0.03


----CN Tower----
                venue  freq
0  Italian Restaurant  0.08
1  

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Hotel,Café,Gym,Japanese Restaurant,Restaurant,Salad Place,American Restaurant,Steakhouse,Asian Restaurant
1,Bathurst Quay,Italian Restaurant,Coffee Shop,Café,Bar,Park,French Restaurant,Gym / Fitness Center,Speakeasy,Lounge,Bakery
2,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market,Restaurant,Breakfast Spot,Cheese Shop,Beer Bar,Pharmacy
3,Brockton,Bar,Café,Coffee Shop,Restaurant,Bakery,Nightclub,Gift Shop,Sandwich Place,Burrito Place,French Restaurant
4,Business reply mail Processing Centre,Coffee Shop,Hotel,Café,Restaurant,Asian Restaurant,Japanese Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Pub


 ### Deliverable 3 - Clustering and Visualization of Toronto Neighborhoods

Run _k_-means to cluster the neighborhood into 5 clusters.


In [33]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 4, 1, 1], dtype=int32)

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = tor_temp3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park,43.65512,-79.36264,1,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Bakery,Greek Restaurant,Italian Restaurant,Food Truck,Event Space,Electronics Store
1,M5A,Downtown Toronto,Harbourfront,43.65512,-79.36264,1,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Bakery,Greek Restaurant,Italian Restaurant,Food Truck,Event Space,Electronics Store
2,M7A,Downtown Toronto,Queen's Park,43.66253,-79.39188,1,Coffee Shop,Park,Theater,Café,Falafel Restaurant,Sandwich Place,Fried Chicken Joint,Mediterranean Restaurant,Italian Restaurant,Gastropub
3,M7A,Downtown Toronto,Ontario Provincial Government,43.66253,-79.39188,1,Coffee Shop,Park,Theater,Café,Falafel Restaurant,Sandwich Place,Fried Chicken Joint,Mediterranean Restaurant,Italian Restaurant,Gastropub
4,M5B,Downtown Toronto,Garden District,43.65739,-79.37804,1,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Café,Hotel,Tanning Salon,Diner,Movie Theater


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters


#### Cluster 1

In [38]:
x1=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
x1['1st Most Common Venue'].value_counts().idxmax()

'Light Rail Station'

In [ ]:
#### Cluster 2

In [39]:
x2=toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
x2['1st Most Common Venue'].value_counts().idxmax()

'Coffee Shop'

In [ ]:
#### Cluster 3

In [40]:
x3=toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
x3['1st Most Common Venue'].value_counts().idxmax()

'Home Service'

#### Cluster 4

In [41]:
x4=toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
x4['1st Most Common Venue'].value_counts().idxmax()

'Park'

#### Cluster 5

In [42]:
x5=toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
x5['1st Most Common Venue'].value_counts().idxmax()

'Furniture / Home Store'

In [43]:
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    xtemp=toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
    new_cluster=xtemp['1st Most Common Venue'].value_counts().idxmax()
    label = folium.Popup(str(new_cluster) + ' Cluster: ' + str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters